In [4]:
!pip install natasha

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 81.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=937a328d0e89cb4b77cbc1d2cff54b49ff4cf7915486f6ec29178801078ceab4
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26094 sha256=fdf9041f012f5d9e0adbd46316745251741f94cab49d95b9f57dacb2074c37c1
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built docopt intervaltree


In [11]:
!pip install fake_useragent

In [1]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [50]:
from natasha import Segmenter, NewsEmbedding, NewsMorphTagger, MorphVocab, Doc

In [2]:
from collections import Counter

In [3]:
import numpy as np

In [5]:
from bs4 import BeautifulSoup
import re

In [6]:
import requests
from pprint import pprint

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from tqdm import tqdm
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
from fake_useragent import UserAgent

In [13]:
from scipy import stats as st

In [14]:
from pymorphy2 import MorphAnalyzer

In [15]:
import pandas as pd

In [16]:
# имитируем пользователя и скачиваем основную страницу со списками отелей с сайта 101hotels.com
ua = UserAgent()
headers = {'User-Agent': ua.random}
session = requests.session()
response = session.get('https://m.101hotels.com/main/cities/moskva/inexpensive', headers=headers)
page = response.text
soup = BeautifulSoup(page, 'html.parser')
looking_for_reviews = soup.find_all('div', {'class': 'hotel__title pr-5'})

In [17]:
# скачиваем отзывы, размеченные как позитивные, они выведены на страницах отелей в разделе "что гостям понравилось"
reviews_positive = []
for item in looking_for_reviews:
    link = item.find('a').attrs['href']
    full_link = 'https://m.101hotels.com' + link
    fragm = session.get(full_link, headers=headers).text
    soup_fragm = BeautifulSoup(fragm, 'html.parser')
    reviews_fragm = soup_fragm.find_all('div', {'class': 'hotel-top-reviews__text'})
    for elem in reviews_fragm:
        review = elem.find('div', {'class': 'response__text-full'}).text
        reviews_positive.append(review)

In [18]:
# скачиваем негативные отзывы, их можно найти в разделе всех отзывов под меткой "response__text response__text_dislike"
reviews_negative = []
for item in looking_for_reviews:
    link = item.find('a').attrs['href']
    full_link = 'https://m.101hotels.com' + link
    fragm = session.get(full_link, headers=headers).text
    soup_fragm = BeautifulSoup(fragm, 'html.parser')
    reviews_link = 'https://m.101hotels.com' + soup_fragm.find('div', {'class': 'hotel-rating-item__text'}).find('a').attrs['href']
    reviews_page = session.get(reviews_link, headers=headers).text
    reviews_content = BeautifulSoup(reviews_page, 'html.parser')
    negative_options = reviews_content.select_one('div', {'class': 'response__content'}).find_all('div', {'class': 'response__text response__text_dislike'})
    for i in negative_options:
        reviews_negative.append(i.text)

In [19]:
# создаем баланс классов
if len(reviews_negative) > len(reviews_positive):
    reviews_negative = reviews_negative[:len(reviews_positive)]
else:
    reviews_positive = reviews_positive[:len(reviews_negative)]

In [20]:
# создаем датасет с отзывами
data = {}
data['text'] = []
data['target'] = []
for post in reviews_positive:
    data['text'].append(post)
    data['target'].append(1)
for post_neg in reviews_negative:
    data['text'].append(post_neg)
    data['target'].append(0)
df = pd.DataFrame(data)
df = shuffle(df)
df.reset_index(drop=True, inplace=True)

In [21]:
# выводим пример того, как выглядит датасет
df

,text,target
0,Хотелось бы более качественную уборку и возмож...,0
1,"В номере почему-то пахло плесенью, номер требу...",0
2,"Расположение прекрасное, чистота, красивый диз...",1
3,"Несоответствие между тем, что указано на сайте...",0
4,"Расположение рядом магазины, кафе, транспорт, ...",1
...,...,...
253,По утрам в туалетах и душевых грязновато. Но у...,0
254,На первый взгляд везде чисто - и в комнатах и ...,1
255,"Отель находится в самом центре, дошли пешком с...",1
256,Не понравилось не очень контактный персонал.За...,0


In [22]:
# сохраняем датасет в файл на будущее
df.to_csv('pos_neg_posts_df.csv')

In [24]:
morph = MorphAnalyzer()
stop_w = set(stopwords.words('russian'))

In [25]:
# создаем функцию для препроцессинга отзывов
def preprocessing(arr):
    corpus=[]
    for text in tqdm(arr):
        text = text.replace('\r\n', ' ')
        words=[word.lower() for word in word_tokenize(text) if ((word.isalpha() == 1) & (word not in stop_w))]
        lemmatized_output = ' '.join([morph.parse(w)[0].normal_form for w in words])
        corpus.append(lemmatized_output)
    return corpus

In [26]:
# применяем функцию к датасету, разбиваем датасет на признаки и целевую переменную
df['text'] = preprocessing(df['text'])
df_x = df['text']
df_y = df['target']

100%|██████████| 258/258 [00:01<00:00, 216.59it/s]


In [27]:
# разбиваем датасет на тренировочную и тестовую выборку
np.random.seed(11)
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.3, random_state=10)

In [28]:
# делаем список только позитивных и только негативных отзывов из датафрейма, смотрим пересечение, смотрим частотность
reviews_positive_train = list(X_train[y_train == 1])
reviews_negative_train = list(X_train[y_train == 0])

positive_words = (' '.join(reviews_positive_train)).split()
dict_positive_apppear = Counter(positive_words)
negative_words = (' '.join(reviews_negative_train)).split()
dict_negative_apppear = Counter(negative_words)

set_positive_words = set(positive_words)
set_negative_words = set(negative_words)
in_both_types = set_positive_words & set_negative_words

list_positive_words = list(set_positive_words)
list_negative_words = list(set_negative_words)
list_in_both_types = list(in_both_types)

In [29]:
# убираем пересечение слов негативных и позитивных отзывов из обоих списков
i = 0
j = 0
while i < len(list_positive_words):
    if list_positive_words[i] in list_in_both_types:
        list_positive_words.pop(i)
    else:
        i += 1

while j < len(list_negative_words):
    if list_negative_words[j] in list_in_both_types:
        list_negative_words.pop(j)
    else:
        j += 1


In [30]:
# пробуем учесть частотности слов и их длину (так могут исключиться вещи, которые не вошли в стоп-слова, но могут ими считаться)
t = 0
k = 0
while t < len(list_positive_words):
    if dict_positive_apppear[list_positive_words[t]] < 3 or len(list_positive_words[t]) < 3:
        list_positive_words.pop(t)
    else:
        t += 1

while k < len(list_negative_words):
    if dict_negative_apppear[list_negative_words[k]] < 3 or len(list_negative_words[k]) < 3:
        list_negative_words.pop(k)
    else:
        k += 1

In [31]:
# создаем итоговые множества позитивных и негативных слов из отзывов
only_positive = set(list_positive_words)
only_negative = set(list_negative_words)

In [32]:
# создаем функцию классификации отзывов по имеющимся множествам позитивных и негативных слов из отзывов
def predict_tone(arr, pos_l, neg_l):
    ans = []
    for text in arr:
        content = set(text.split())
        # считаем, каких вхождений больше, выводим превалирующее
        if len(content & pos_l) > len(content & neg_l):
            ans.append(1)
        else:
            ans.append(0)
    return ans

In [33]:
# создаем функцию для подсчета accuracy
def accuracy_score(prediction, y):
    return (len(prediction) - np.sum(np.absolute(np.array(prediction) - np.array(y)))) / len(prediction)

In [34]:
# делаем предсказание на тренировочной и тестовых выборках
predicted_train = predict_tone(X_train, only_positive, only_negative)
predicted_test = predict_tone(X_test, only_positive, only_negative)

In [35]:
# выводим accuracy для тренировочной и тестовой выборок
print(f"train = {accuracy_score(predicted_train, y_train)}")
print(f"test = {accuracy_score(predicted_test, y_test)}")

train = 0.9833333333333333
test = 0.8461538461538461


#### Как улучшить алгоритм:

1. попробуем использовать биграммы вместо одиночных слов
   
2. векторизация текста и подсчет евклидовой метрики между векторами тренировочной и тестовой выборок

In [36]:
# реализация первого способа, функция для поиска биграмм позитивных и негативных отзывов
def bigrams_search(arr):
    bigrams = []
    for text in tqdm(arr):
        text_new = re.findall(r'[^\s]+[\s][^\s]+', text)
        for bigram in text_new:
            bigrams.append(bigram)
        text_add = ' '.join(text.split()[1:])
        text_add_new = re.findall(r'[^\s]+[\s][^\s]+', text_add)
        for bigram_add in text_add_new:
            bigrams.append(bigram_add)
    return bigrams

In [38]:
bigrams_positive = bigrams_search(X_train[y_train == 1])
bigrams_negative = bigrams_search(X_train[y_train == 0])

100%|██████████| 98/98 [00:00<00:00, 21962.05it/s]


In [39]:
# смотрим на частотность биграмм, создаем список биграмм позитивных и негативных отзывов, смотрим пересечение
dict_positive_bigr = Counter(bigrams_positive)
dict_negative_bigr = Counter(bigrams_negative)

set_positive_bigr = set(bigrams_positive)
set_negative_bigr = set(bigrams_negative)
in_both_bigr = set_positive_bigr & set_negative_bigr

list_positive_bigr = list(set_positive_bigr)
list_negative_bigr = list(set_negative_bigr)
list_in_both_bigr = list(in_both_bigr)

In [40]:
#  убираем пересечение из списков, чистим списки по частотности
f = 0
l = 0
while f < len(list_positive_bigr):
    if list_positive_bigr[f] in list_in_both_bigr or dict_positive_bigr[list_positive_bigr[f]] < 3:
        list_positive_bigr.pop(f)
    else:
        f += 1

while l < len(list_negative_bigr):
    if list_negative_bigr[l] in list_in_both_bigr or dict_negative_bigr[list_negative_bigr[l]] < 3:
        list_negative_bigr.pop(l)
    else:
        l += 1

In [41]:
only_positive_bigr = set(list_positive_bigr)
only_negative_bigr = set(list_negative_bigr)

In [43]:
# функция для предсказания тональности по биграммам
def predict_tone_bigr(arr, pos_l, neg_l):
    ans = []
    for text in arr:
        content = set(re.findall(r'[^\s]+[\s][^\s]+', text)) | set(re.findall(r'[^\s]+[\s][^\s]+', ' '.join(text.split()[1:])))
        if len(content & pos_l) > len(content & neg_l):
            ans.append(1)
        else:
            ans.append(0)
    return ans

In [44]:
# предсказания на тренировочной и тестовой выборках
predicted_train_bigr = predict_tone_bigr(X_train, only_positive_bigr, only_negative_bigr)
predicted_test_bigr = predict_tone_bigr(X_test, only_positive_bigr, only_negative_bigr)

In [45]:
# выводим accuracy для тренировочной и тестовой выборок
print(f"train = {accuracy_score(predicted_train_bigr, y_train)}")
print(f"test = {accuracy_score(predicted_test_bigr, y_test)}")

train = 0.8333333333333334
test = 0.6794871794871795


In [46]:
# реализация второго способа
vect = CountVectorizer(min_df = 10)
X_train_bow = vect.fit_transform(X_train).toarray()
X_test_bow = vect.transform(X_test).toarray()

In [47]:
# функция для предсказания целевой переменной по векторам
def predict_new_way(k, y_train, X_train, X_test):
    # сортирую полученные значения, выводя их индексы, оставляю k нужных
    # здесь я с помощью евклидовой метрики вычисляю расстояние между объектами тестовой и тренировочной выборки
    # потом сортирую полученные значения, выводя их индексы, оставляю k нужных
    index = np.argsort(((((X_test[:, np.newaxis, :] - X_train[np.newaxis, :, :])**2).sum(axis=2))**0.5), axis = -1)[:, :k]
    # вывожу моду среди значений по индексам
    return st.mode(np.array(y_train)[index], axis=1).mode

In [48]:
# предсказание на тестовой выборке
predicted_new_way = predict_new_way(3, y_train, X_train_bow, X_test_bow)

In [49]:
# выводим accuracy для тестовой выборки
print(accuracy_score(predicted_new_way, y_test))

0.5897435897435898


#### Chunker!!!

In [51]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [52]:
def chunker(sequence, size):
    output = []
    for elem in range(0, len(sequence) - (size - 1)):
        output.append(sequence[elem: elem + size])
    return output

In [53]:
def bigrams(text):
    result = []
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for sht in chunker(doc.tokens, 2):
        if sht[0].pos == 'ADV' and sht[1].pos == 'ADJ':
            result.append(' '.join([sht[0].text, sht[1].text]))
        if sht[0].pos == 'ADJ' and sht[1].pos == 'NOUN':
            result.append(' '.join([sht[0].text, sht[1].text]))
    for sht in chunker(doc.tokens, 3):
        if sht[0].text == 'не' and sht[1].pos == 'VERB':
            result.append(' '.join([sht[0].text, sht[1].text]))
    return set(result)

In [58]:
bigrams_positive = bigrams(' '.join([x for x in X_train[y_train == 1]]))
bigrams_negative = bigrams(' '.join([x for x in X_train[y_train == 0]]))
total_positive = only_positive | bigrams_positive
total_negative = only_negative | bigrams_negative

In [59]:
ans = []
arr = [x for x in X_test]
for text in arr:
  content = bigrams(text) | set(text.split())
  if len(content & total_positive) > len(content & total_negative):
    ans.append(1)
  else:
    ans.append(0)

In [60]:
print(accuracy_score(ans, y_test))

0.8717948717948718


Качество предсказания с улучшением стало лучше! Было 0,84 (см самое первое предсказание на тесте), а стало 0,87. Я улучшение прикрутила к самому хорошему варианту, то есть к сету уникальных слов добавила наши биграммы, полученные с помощью чанкера.